In [1]:
import os

os.chdir('../')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [5]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


# For the sake of speed let's use topK=100 for both

In [ ]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender
import gc
from Recommenders.Recommender_utils import similarityMatrixTopK

best_similarity = sps.load_npz("SimilarityMatrixes/merging_models_final_similarity_TRAINVAL.npz")
best_similarity = similarityMatrixTopK(best_similarity, k=100, use_absolute_values=True)
gc.collect()
similarity_rec = ItemKNNCustomSimilarityRecommender(URM_train_validation)
similarity_rec.fit(best_similarity)

IALS_recommender = ImplicitALSRecommender(URM_train_validation)
IALS_recommender.fit(factors=1903, regularization=0.43781149051372426, iterations=10, use_gpu=False, **{"alpha": 10.365656056877015})
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution
W_graph_convolution = sps.load_npz("SimilarityMatrixes/graphConvolution_W_sparse_TRAINVAL_num_factors:5645_alpha:_0.223329.npz")
graph_recommender = GraphConvolution(URM_train_validation)

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

AttributeError: 'GraphConvolution' object has no attribute 'W_sparse'

In [7]:
graph_recommender.W_sparse = similarityMatrixTopK(W_graph_convolution, k=100, use_absolute_values=True)
gc.collect()

488

In [8]:
new_recommenders = [similarity_rec, IALS_recommender, graph_recommender]
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender
def objective_function(trial):

    raw_alphas = [trial.suggest_float(f'alpha_{i}', 0, 1) for i in range(len(new_recommenders))]
    
    # Normalizza per garantire che la somma sia esattamente 1
    alphas = np.array(raw_alphas) / sum(raw_alphas)
    print(alphas)
    # Salva le alphas normalizzate come attributo del trial
    trial.set_user_attr("normalized_alphas", alphas.tolist())
    final_recommender = GeneralizedLinearHybridRecommender(URM_train_validation, new_recommenders)
    final_recommender.fit(**{"alphas": alphas})
    result_dict, _ = evaluator_test.evaluateRecommender(final_recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/linear_comb_similarity_IALS_GRAPH.csv", index = False)
        
import optuna

optuna_study = optuna.create_study(study_name="linear_comb", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2025-01-04 12:13:22,877] A new study created in memory with name: linear_comb


[0.05651724 0.40896517 0.53451759]
EvaluatorHoldout: Processed 35595 (100.0%) in 38.09 sec. Users per second: 934


[I 2025-01-04 12:14:01,023] Trial 0 finished with value: 0.052981731204443244 and parameters: {'alpha_0': 0.09738646076365964, 'alpha_1': 0.7046995401196545, 'alpha_2': 0.9210425044634367}. Best is trial 0 with value: 0.052981731204443244.


[0.27474705 0.62311109 0.10214185]
EvaluatorHoldout: Processed 35595 (100.0%) in 37.31 sec. Users per second: 954


[I 2025-01-04 12:14:38,368] Trial 1 finished with value: 0.05518643380000133 and parameters: {'alpha_0': 0.3684681293954496, 'alpha_1': 0.8356653021572328, 'alpha_2': 0.1369842421062364}. Best is trial 1 with value: 0.05518643380000133.


...

[0.78657114 0.1907694  0.02265946]
EvaluatorHoldout: Processed 35595 (100.0%) in 37.19 sec. Users per second: 957


[I 2025-01-04 13:03:08,014] Trial 81 finished with value: 0.061006115982936815 and parameters: {'alpha_0': 0.9684830015073798, 'alpha_1': 0.23488901468715598, 'alpha_2': 0.02789996046493383}. Best is trial 27 with value: 0.06110140981990795.


[0.73902129 0.19191828 0.06906043]


[W 2025-01-04 13:03:43,510] Trial 82 failed with parameters: {'alpha_0': 0.8892508959592103, 'alpha_1': 0.2309317803790319, 'alpha_2': 0.08309915929590303} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_46733/3714798673.py", line 14, in objective_function
    result_dict, _ = evaluator_test.evaluateRecommender(final_recommender)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = r

KeyboardInterrupt: 